In [7]:
from hydra import initialize, compose
from pathlib import Path
import pandas as pd
from ergochemics.draw import draw_reaction, draw_molecule
from ergochemics.mapping import rc_to_nest

with initialize(version_base=None, config_path="./conf/filepaths"):
    cfg = compose(config_name="filepaths")

In [8]:
krs = pd.read_parquet(Path(cfg.processed_data) / "pathway" / "known_reactions.parquet")
tot_krs = len(krs)
print(f"Total known reactions: {tot_krs}")

Total known reactions: 28142


In [9]:
rules = [
    "mechinformed",
    # "mechinferred_dt_",
    "rc_plus_0",
    # "rc_plus_1",
    # "rc_plus_2",
    # "rc_plus_3",
    # "rc_plus_4",
]


for rule in rules:
    fn = f"mapped_known_reactions_x_{rule}_rules.parquet"
    df = pd.read_parquet(Path(cfg.processed_data) / "pathway" / fn)
    df["template_aidxs"] = df["template_aidxs"].apply(rc_to_nest)
    print(len(df))
    print(f"{rule} kr coverage: {len(df) * 100 / tot_krs:.1f}%")

12137
mechinformed kr coverage: 43.1%
19433
rc_plus_0 kr coverage: 69.1%
